In [1]:
import numpy as np
import pandas as pd

In [83]:
class Lab3(object):
    def __init__(self, start_condition_function, x_equation, y_equation, start_x, end_x, step, second_equation_value, end_y):
        self._x_values = self.get_x_values(start_x, end_x, step)
        self._end_x = end_x
        self._start_condition_function = start_condition_function
        self._step = step
        self._x_equation = x_equation
        self._y_eqution = y_equation
        self._second_equation_value = second_equation_value
        self._end_y = end_y
    
    def get_x_values(self, start_x, end_x, step):
        return np.arange(start_x, end_x, step)
    
    def _get_left_first_derivative_coefficients(self):
        return np.array([-1./self._step, 1./self._step])
    
    def get_first_equation(self):
        return self._start_condition_function(self._get_left_first_derivative_coefficients)
    
    def _get_mid_first_equation(self):
        return np.array([-1./(2.*self._step), 0, 1./(2.*self._step)])
    
    def _get_second_derivative_coefficients(self):
        return np.array([1./self._step**2, -2./self._step**2, 1./self._step**2])
    
    def _get_y_coefficients(self):
        return np.array([0., 1., 0.])
    
    def get_equation(self, x_value):
        second_derivative_coefficients = self._get_second_derivative_coefficients()
        first_derivative_value = self._x_equation(x_value) * self._get_mid_first_equation()
        y_equation_coefficients = self._y_eqution(x_value) * self._get_y_coefficients()
#         print("----------------------")
#         print(second_derivative_coefficients)
#         print(first_derivative_value)
#         print(y_equation_coefficients)
#         print("----------------------")
        return second_derivative_coefficients + first_derivative_value + y_equation_coefficients, self._second_equation_value
    
    def get_second_equation(self):
        return self.get_equation(self._x_values[1])
    
    def get_third_equation(self):
        coefficients, value = self.get_equation(self._x_values[2])
        value -= coefficients[2]*self._end_y
        return coefficients[:-1], value
    
    def main(self):
        coefficients, value = self.get_first_equation()
        print("{:.1f}Y0+{:.1f}Y1 = {:.1f}".format(coefficients[0], coefficients[1], value))
        coefficients, value = self.get_second_equation()
        print("{:.1f}Y0{:.2f}Y1 + {:.1f}Y2 = {:.1f}".format(coefficients[0], coefficients[1], coefficients[2], value))
        coefficients, value = self.get_third_equation()
        print("{:.1f}Y0{:.2f}Y1 = {:.1f}".format(coefficients[0], coefficients[1], value))

In [84]:
def start_condition(derivative):
    return 2*derivative() + np.array([1., 0.]), 1.

def x_equation(x):
    return x

def y_equation(x):
    return -0.5/x

start_x = 2.
end_x = 2.3
end_y = 2.15
function_value = 1.
step = 0.1
test = Lab3(start_condition, x_equation, y_equation, start_x, end_x, step, function_value, end_y)
test.main()

-19.0Y0+20.0Y1 = 1.0
89.5Y0-200.24Y1 + 110.5Y2 = 1.0
89.0Y0-200.23Y1 = -237.6
